<a href="https://colab.research.google.com/github/Imran2205/uwsnet/blob/main/colab_notebooks/fewshot_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install gdown

In [2]:
![[ -d ./uwsnet ]] && rm -rf ./uwsnet
! git clone https://github.com/Imran2205/uwsnet

Cloning into 'uwsnet'...
remote: Enumerating objects: 638, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 638 (delta 9), reused 6 (delta 1), pack-reused 619
Receiving objects: 100% (638/638), 1.60 MiB | 10.03 MiB/s, done.
Resolving deltas: 100% (387/387), done.


In [5]:
# download fewshot dataset from google drive and extract it
! gdown https://drive.google.com/uc?id=1tmf7179sRqrxIlmJprDpj9h5jcBLA7k1
! unzip -q uw_fewshot_dataset.zip
# ! rm -rf /content/uwsnet/dataset/*
# ! mv dataset/* /content/uwsnet/dataset
! rm -rf /content/__MACOSX
# ! rm -rf /content/dataset
! rm -rf /content/uw_fewshot_dataset.zip

Downloading...
From: https://drive.google.com/uc?id=1tmf7179sRqrxIlmJprDpj9h5jcBLA7k1
To: /content/uw_fewshot_dataset.zip
100% 917M/917M [00:15<00:00, 60.2MB/s]


In [4]:
# download dataset splits
import gdown
gdown.download_folder('https://drive.google.com/drive/folders/1IOLhsgBd2ZnVr4I10s8m4PqfP2yzpEbp', quiet=True)

['/content/dataset_split/classes2labels.joblib',
 '/content/dataset_split/labels_split.joblib']

In [6]:
! rm -rf /content/uwsnet/dataset_split/*
! mv dataset_split/* /content/uwsnet/dataset_split
! rm -rf /content/dataset_split

In [7]:
# download backbones
gdown.download_folder('https://drive.google.com/drive/folders/15AD86KAKEpySFu8NFNZcPWE_9vmAuGx5', quiet=True)

['/content/backbones/resnet50_v2.pth',
 '/content/backbones/resnet101_v2.pth',
 '/content/backbones/vgg16_bn.pth',
 '/content/backbones/vgg16-397923af.pth']

In [8]:
# download pretrained models
gdown.download_folder('https://drive.google.com/drive/folders/1WEtRqBZgHtteCIzFHhnutp_KsXGA2Dhh', quiet=True)

['/content/fewshot_trained_model/UWSNetV6/shot_1/split_1/best_model.pth.tar',
 '/content/fewshot_trained_model/UWSNetV6/shot_1/split_2/best_model.pth.tar',
 '/content/fewshot_trained_model/UWSNetV6/shot_1/split_3/best_model.pth.tar',
 '/content/fewshot_trained_model/UWSNetV6/shot_1/split_4/best_model.pth.tar',
 '/content/fewshot_trained_model/UWSNetV6/shot_5/split_1/best_model.pth.tar',
 '/content/fewshot_trained_model/UWSNetV6/shot_5/split_2/best_model.pth.tar',
 '/content/fewshot_trained_model/UWSNetV6/shot_5/split_3/best_model.pth.tar',
 '/content/fewshot_trained_model/UWSNetV6/shot_5/split_4/best_model.pth.tar']

In [9]:
! rm -rf log
! rm -rf output

In [10]:
! mkdir -p log
! mkdir -p output

In [11]:
! pip install imagecodecs
! pip install pickle5
! pip install timm
! pip install einops
! pip install pytorch_lightning
! pip install yacs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.1/132.1 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pickle5: filename=pickle5-0.0.11-cp310-cp310-linux_x86_64.whl size=256387 sha256=1a9ba11bf146d802e0214b559568bbc0f6bf30267a006b023e7a4374f86eeb9d
  Stored in directory: /root/.cache/pip/wheels/7d/14/ef/4aab19d27fa8e58772be5c71c16add0426acf9e1f64353235c
Successfully built pickle5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 722.4/722.4 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 729.2/729.2 kB 58.3 MB/s eta 0:00:00


In [12]:
# Inference of UWSNet V2 for 1 shot with split 1 as validation set
# ! python3 ./uwsnet/tools/test.py \
#     --config ./uwsnet/experiments_inference/eca_net_sup_que_vgg16_config/1_shot/uw_few_shot_training_config_eca_net_sup_que_vgg16_shot_1_split_0.yaml

# Inference of UWSNet V2 for 1 shot with split 2 as validation set
# ! python3 ./uwsnet/tools/test.py \
#     --config ./uwsnet/experiments_inference/eca_net_sup_que_vgg16_config/1_shot/uw_few_shot_training_config_eca_net_sup_que_vgg16_shot_1_split_1.yaml

# Inference of UWSNet V2 for 1 shot with split 3 as validation set
# ! python3 ./uwsnet/tools/test.py \
#     --config ./uwsnet/experiments_inference/eca_net_sup_que_vgg16_config/1_shot/uw_few_shot_training_config_eca_net_sup_que_vgg16_shot_1_split_2.yaml

# Inference of UWSNet V2 for 1 shot with split 4 as validation set
# ! python3 ./uwsnet/tools/test.py \
#     --config ./uwsnet/experiments_inference/eca_net_sup_que_vgg16_config/1_shot/uw_few_shot_training_config_eca_net_sup_que_vgg16_shot_1_split_3.yaml

# Inference of UWSNet V6 for 1 shot with split 1 as validation set
! python3 ./uwsnet/tools/test.py \
    --config ./uwsnet/experiments_inference/triplet_sup_que_vgg16_dice_config/1_shot/uw_few_shot_training_config_triplet_sup_que_vgg16_dice_shot_1_split_0.yaml

# Inference of UWSNet V6 for 1 shot with split 2 as validation set
# ! python3 ./uwsnet/tools/test.py \
#     --config ./uwsnet/experiments_inference/triplet_sup_que_vgg16_dice_config/1_shot/uw_few_shot_training_config_triplet_sup_que_vgg16_dice_shot_1_split_1.yaml

# Inference of UWSNet V6 for 1 shot with split 3 as validation set
# ! python3 ./uwsnet/tools/test.py \
#     --config ./uwsnet/experiments_inference/triplet_sup_que_vgg16_dice_config/1_shot/uw_few_shot_training_config_triplet_sup_que_vgg16_dice_shot_1_split_2.yaml

# Inference of UWSNet V6 for 1 shot with split 4 as validation set
# ! python3 ./uwsnet/tools/test.py \
#     --config ./uwsnet/experiments_inference/triplet_sup_que_vgg16_dice_config/1_shot/uw_few_shot_training_config_triplet_sup_que_vgg16_dice_shot_1_split_3.yaml

2023-07-13 14:30:20.822271: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-13 14:30:22.141616: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Using PyTorch version:  2.0.1+cu118
PAnet model will be used ......
=> creating /content/output/uw_few_shot_training_config_triplet_sup_que_vgg16_dice_shot_1_split_0
=> creating /content/log/PAnet/uw_few_shot_training_config_triplet_sup_que_vgg16_dice_shot_1_split_0_2023-07-13-14-30
BASE_DIR: /content/uwsnet/
CLASSES: 2
CUDNN:
  BENCHMARK: True
  DETERMINISTIC: False
  ENABLED: True
DATASET:
  IMAGE_H: 473
  IMAGE_W: 473
  ROOT: /content/
DATA_DIR: 
DEVICE: cuda
GPU: 0
LOG_DIR: /content/log/
MODEL:
  INIT_WEIGHTS: True
  NAME: PAnet
  PRETRAINED: 
OUTPUT_DIR

In [13]:
! python3 ./uwsnet/tools/test.py \
    --config ./uwsnet/experiments_inference/triplet_sup_que_vgg16_dice_config/1_shot/uw_few_shot_training_config_triplet_sup_que_vgg16_dice_shot_1_split_1.yaml

2023-07-13 14:38:16.708438: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-13 14:38:18.073208: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Using PyTorch version:  2.0.1+cu118
PAnet model will be used ......
=> creating /content/output/uw_few_shot_training_config_triplet_sup_que_vgg16_dice_shot_1_split_1
=> creating /content/log/PAnet/uw_few_shot_training_config_triplet_sup_que_vgg16_dice_shot_1_split_1_2023-07-13-14-38
BASE_DIR: /content/uwsnet/
CLASSES: 2
CUDNN:
  BENCHMARK: True
  DETERMINISTIC: False
  ENABLED: True
DATASET:
  IMAGE_H: 473
  IMAGE_W: 473
  ROOT: /content/
DATA_DIR: 
DEVICE: cuda
GPU: 0
LOG_DIR: /content/log/
MODEL:
  INIT_WEIGHTS: True
  NAME: PAnet
  PRETRAINED: 
OUTPUT_DIR

In [14]:
! python3 ./uwsnet/tools/test.py \
    --config ./uwsnet/experiments_inference/triplet_sup_que_vgg16_dice_config/1_shot/uw_few_shot_training_config_triplet_sup_que_vgg16_dice_shot_1_split_2.yaml

2023-07-13 14:47:42.958872: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-13 14:47:44.503088: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Using PyTorch version:  2.0.1+cu118
PAnet model will be used ......
=> creating /content/output/uw_few_shot_training_config_triplet_sup_que_vgg16_dice_shot_1_split_2
=> creating /content/log/PAnet/uw_few_shot_training_config_triplet_sup_que_vgg16_dice_shot_1_split_2_2023-07-13-14-47
BASE_DIR: /content/uwsnet/
CLASSES: 2
CUDNN:
  BENCHMARK: True
  DETERMINISTIC: False
  ENABLED: True
DATASET:
  IMAGE_H: 473
  IMAGE_W: 473
  ROOT: /content/
DATA_DIR: 
DEVICE: cuda
GPU: 0
LOG_DIR: /content/log/
MODEL:
  INIT_WEIGHTS: True
  NAME: PAnet
  PRETRAINED: 
OUTPUT_DIR

In [ ]:
! python3 ./uwsnet/tools/test.py \
    --config ./uwsnet/experiments_inference/triplet_sup_que_vgg16_dice_config/1_shot/uw_few_shot_training_config_triplet_sup_que_vgg16_dice_shot_1_split_3.yaml

In [16]:
! python3 ./uwsnet/tools/test.py \
    --config ./uwsnet/experiments_inference/triplet_sup_que_vgg16_dice_config/5_shot/uw_few_shot_training_config_triplet_sup_que_vgg16_dice_shot_5_split_0.yaml

2023-07-13 14:56:55.193736: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-13 14:56:56.603857: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Using PyTorch version:  2.0.1+cu118
PAnet model will be used ......
=> creating /content/output/uw_few_shot_training_config_triplet_sup_que_vgg16_dice_shot_5_split_0
=> creating /content/log/PAnet/uw_few_shot_training_config_triplet_sup_que_vgg16_dice_shot_5_split_0_2023-07-13-14-56
BASE_DIR: /content/uwsnet/
CLASSES: 2
CUDNN:
  BENCHMARK: True
  DETERMINISTIC: False
  ENABLED: True
DATASET:
  IMAGE_H: 473
  IMAGE_W: 473
  ROOT: /content/
DATA_DIR: 
DEVICE: cuda
GPU: 0
LOG_DIR: /content/log/
MODEL:
  INIT_WEIGHTS: True
  NAME: PAnet
  PRETRAINED: 
OUTPUT_DIR

In [ ]:
! python3 ./uwsnet/tools/test.py \
    --config ./uwsnet/experiments_inference/triplet_sup_que_vgg16_dice_config/5_shot/uw_few_shot_training_config_triplet_sup_que_vgg16_dice_shot_5_split_1.yaml

2023-07-13 15:12:13.644779: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-13 15:12:14.652787: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Using PyTorch version:  2.0.1+cu118
PAnet model will be used ......
=> creating /content/output/uw_few_shot_training_config_triplet_sup_que_vgg16_dice_shot_5_split_1
=> creating /content/log/PAnet/uw_few_shot_training_config_triplet_sup_que_vgg16_dice_shot_5_split_1_2023-07-13-15-12
BASE_DIR: /content/uwsnet/
CLASSES: 2
CUDNN:
  BENCHMARK: True
  DETERMINISTIC: False
  ENABLED: True
DATASET:
  IMAGE_H: 473
  IMAGE_W: 473
  ROOT: /content/
DATA_DIR: 
DEVICE: cuda
GPU: 0
LOG_DIR: /content/log/
MODEL:
  INIT_WEIGHTS: True
  NAME: PAnet
  PRETRAINED: 
OUTPUT_DIR

In [ ]:
! python3 ./uwsnet/tools/test.py \
    --config ./uwsnet/experiments_inference/triplet_sup_que_vgg16_dice_config/5_shot/uw_few_shot_training_config_triplet_sup_que_vgg16_dice_shot_5_split_2.yaml

In [ ]:
! python3 ./uwsnet/tools/test.py \
    --config ./uwsnet/experiments_inference/triplet_sup_que_vgg16_dice_config/5_shot/uw_few_shot_training_config_triplet_sup_que_vgg16_dice_shot_5_split_3.yaml